In [2]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [3]:
# hostip = "192.168.1.87"
# hostip = "10.192.40.238"
hostip = "192.168.0.88"
topic = 'ProducerC'
file_path = 'A2/data/camera_event_C.csv' 


def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
        

In [4]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

In [5]:
if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Failed to connect to Kafka. Exiting..')
        exit(1)

    current_batch = None
    batch_events = []

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_row = dumps(row)
            if current_batch is None:
                current_batch = row['batch_id']

            if current_batch != row['batch_id']:
                for event in batch_events:
                    message = {
                        'event_id': event['event_id'],
                        "car_plate": event['car_plate'],
                        "camera_id": event['camera_id'],
                        "timestamp": event['timestamp'],
                        "speed_reading": float(event['speed_reading']),
                        "producer": "C"
                    }
                    publish_message(producer, topic, event['event_id'], dumps(message))
                producer.flush()
                print(f'Batch {current_batch} published successfully.')
                sleep(5)

                current_batch = row['batch_id']
                batch_events = []

            batch_events.append(row)

        if batch_events:
            for event in batch_events:
                message = {
                    'event_id': event['event_id'],
                    "car_plate": event['car_plate'],
                    "camera_id": event['camera_id'],
                    "timestamp": event['timestamp'],
                    "speed_reading": float(event['speed_reading']),
                    "producer": "C"
                }
                publish_message(producer, topic, event['event_id'], dumps(message))
            producer.flush()
            print(f'Final batch {current_batch} published successfully.')

    print('All messages published successfully.')

Publishing records..
Message published successfully. {"event_id": "d86e8cdb-c387-4ccb-b35a-346302238824", "car_plate": "UTT 229", "camera_id": "3", "timestamp": "2024-01-01T08:00:54.958092", "speed_reading": 130.8, "producer": "C"}
Batch 1 published successfully.
Message published successfully. {"event_id": "f804c4e8-022d-4243-b43e-9ec0860ce6ff", "car_plate": "CJW 924", "camera_id": "3", "timestamp": "2024-01-01T08:00:47.168827", "speed_reading": 164.5, "producer": "C"}
Batch 2 published successfully.
Message published successfully. {"event_id": "a246b391-fb7c-44a3-8f8b-ddf40bba3d05", "car_plate": "WA 0712", "camera_id": "3", "timestamp": "2024-01-01T08:00:53.815270", "speed_reading": 134.4, "producer": "C"}
Message published successfully. {"event_id": "e0dae388-6036-4b7d-ba58-bed0808c1611", "car_plate": "KZE 5", "camera_id": "3", "timestamp": "2024-01-01T08:00:57.613888", "speed_reading": 125.3, "producer": "C"}
Batch 3 published successfully.
Message published successfully. {"event_i

KeyboardInterrupt: 